In [4]:
import pandas as pd
import os
import joblib
import time
import itertools
import warnings
warnings.filterwarnings("ignore")

import pennylane as qml
from pennylane import numpy as np
import pennylane.numpy as pnp

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, roc_auc_score
from sklearn.decomposition import PCA

# --- Load and preprocess data ---
df = pd.read_csv("mergedTop20.csv")
df["Label"] = df["Label"].astype(str).str.strip().str.upper()
df["Label"] = df["Label"].apply(lambda x: 1 if x == "BENIGN" else 0)

X = df.drop("Label", axis=1).values
y = df["Label"].values
y = 2 * y - 1  # to {-1, 1}

n_qubits = 4
pca = PCA(n_components=n_qubits)
X_reduced = pca.fit_transform(X)

scaler = MinMaxScaler(feature_range=(0, 2 * np.pi))
X_scaled = scaler.fit_transform(X_reduced)

# Balance dataset
benign_idx = np.where(y == 1)[0]
attack_idx = np.where(y == -1)[0]
n_samples = min(len(benign_idx), len(attack_idx), 500)

idx = np.concatenate([
    np.random.choice(benign_idx, n_samples, replace=False),
    np.random.choice(attack_idx, n_samples, replace=False)
])
X_bal = X_scaled[idx]
y_bal = y[idx]

X_train, X_test, y_train, y_test = train_test_split(
    X_bal, y_bal, test_size=0.2, stratify=y_bal, random_state=42
)

# --- Define variations ---
circuit_types = ["StronglyEntanglingLayers", "BasicEntanglerLayers"]
optimizers = {
    "Adam": lambda lr: qml.AdamOptimizer(stepsize=lr),
    "Momentum": lambda lr: qml.NesterovMomentumOptimizer(stepsize=lr),
    "SGD": lambda lr: qml.GradientDescentOptimizer(stepsize=lr)
}
init_schemes = {
    "normal_0.01": lambda shape: 0.01 * np.random.randn(*shape),
    "uniform_2pi": lambda shape: np.random.uniform(0, 2 * np.pi, size=shape)
}

def generate_weights(layer_type, init_type, layers, n_qubits):
    if layer_type == "StronglyEntanglingLayers":
        shape = (layers, n_qubits, 3)
    elif layer_type == "BasicEntanglerLayers":
        shape = (layers, n_qubits)
    else:
        raise ValueError(f"Unknown layer type: {layer_type}")

    if init_type == "normal_0.01":
        return 0.01 * np.random.randn(*shape)
    elif init_type == "uniform_2pi":
        return np.random.uniform(0, 2 * np.pi, size=shape)
    else:
        raise ValueError(f"Unknown init_type: {init_type}")

results = []
start_time = time.time()

for (circ_type, opt_name, init_name) in itertools.product(circuit_types, optimizers, init_schemes):
    print(f"\n----- Testing {circ_type} + {opt_name} + {init_name} -----")
    
    dev = qml.device("lightning.qubit", wires=n_qubits)

    def variational_circuit(x, weights):
        qml.AngleEmbedding(x, wires=range(n_qubits), rotation='Y')
        if circ_type == "StronglyEntanglingLayers":
            qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
        elif circ_type == "BasicEntanglerLayers":
            qml.BasicEntanglerLayers(weights, wires=range(n_qubits))

    @qml.qnode(dev)
    def circuit(x, weights):
        variational_circuit(x, weights)
        return qml.expval(qml.PauliZ(0))

    def predict(weights, X):
        raw = [circuit(x, weights) for x in X]
        return np.array([1 if r > 0 else -1 for r in raw]), np.array(raw)

    def cost(weights, X, y):
        losses = [qml.math.square(circuit(x, weights) - y_i) for x, y_i in zip(X, y)]
        return qml.math.mean(pnp.stack(losses))

    # Init
    layers = 3
    shape = (layers, n_qubits, 3)
    weights_init = generate_weights(circ_type, init_name, layers, n_qubits)
    weights = weights_init.copy()
    opt = optimizers[opt_name](0.005)

    # Training
    epochs = 30
    for _ in range(epochs):
        weights = opt.step(lambda w: cost(w, X_train, y_train), weights)

    # Evaluation
    y_pred_bin, y_pred_raw = predict(weights, X_test)
    y_test_bin = ((y_test + 1) // 2).astype(int)
    y_pred_final = ((y_pred_bin + 1) // 2).astype(int)

    acc = accuracy_score(y_test_bin, y_pred_final)
    f1 = f1_score(y_test_bin, y_pred_final)
    auc = roc_auc_score(y_test_bin, y_pred_raw)
    loss = cost(weights, X_train, y_train)

    print(f"Accuracy: {acc:.4f} | F1: {f1:.4f} | AUC: {auc:.4f} | Final Loss: {loss:.4f}")
    results.append((circ_type, opt_name, init_name, acc, f1, auc, float(loss)))

# --- Results Summary ---
print("\n===== Summary of All Runs =====")
results_df = pd.DataFrame(results, columns=["Circuit", "Optimizer", "Init", "Accuracy", "F1", "AUC", "Loss"])
print(results_df.sort_values(by="F1", ascending=False))

print(f"\nTotal time: {round(time.time() - start_time, 2)} seconds")



----- Testing StronglyEntanglingLayers + Adam + normal_0.01 -----
Accuracy: 0.5000 | F1: 0.6667 | AUC: 0.2356 | Final Loss: 1.5543

----- Testing StronglyEntanglingLayers + Adam + uniform_2pi -----
Accuracy: 0.5250 | F1: 0.0952 | AUC: 0.6473 | Final Loss: 0.9176

----- Testing StronglyEntanglingLayers + Momentum + normal_0.01 -----
Accuracy: 0.4800 | F1: 0.6486 | AUC: 0.1763 | Final Loss: 1.0964

----- Testing StronglyEntanglingLayers + Momentum + uniform_2pi -----
Accuracy: 0.6150 | F1: 0.7094 | AUC: 0.6136 | Final Loss: 0.8880

----- Testing StronglyEntanglingLayers + SGD + normal_0.01 -----
Accuracy: 0.5000 | F1: 0.6667 | AUC: 0.1766 | Final Loss: 1.7365

----- Testing StronglyEntanglingLayers + SGD + uniform_2pi -----
Accuracy: 0.4950 | F1: 0.1920 | AUC: 0.5645 | Final Loss: 0.9962

----- Testing BasicEntanglerLayers + Adam + normal_0.01 -----
Accuracy: 0.5000 | F1: 0.6667 | AUC: 0.3124 | Final Loss: 1.4018

----- Testing BasicEntanglerLayers + Adam + uniform_2pi -----
Accuracy: 0